In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('/kaggle/input/crime-cast-forecasting-crime-categories/train.csv')
test_data = pd.read_csv('/kaggle/input/crime-cast-forecasting-crime-categories/test.csv')

In [ ]:
data

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import  OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler,MinMaxScaler

from sklearn.dummy import DummyClassifier

from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.model_selection import learning_curve

from sklearn.pipeline import Pipeline

from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report, accuracy_score


from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.svm import LinearSVC

from sklearn.ensemble import RandomForestClassifier

from xgboost import XGBClassifier

from sklearn.linear_model import LogisticRegression

from sklearn.naive_bayes import GaussianNB

from sklearn.neighbors import KNeighborsClassifier

from sklearn.svm import SVC
from sklearn.svm import LinearSVC

from sklearn.neural_network import MLPClassifier

from sklearn.tree import DecisionTreeClassifier

import category_encoders as ce

In [ ]:
feature_list = data.columns[:-1].values
target = [data.columns[-1]]

In [ ]:
feature = data[feature_list]
label = data[target]

In [ ]:
feature

In [ ]:
label

Data Preprocessing and cleaning

In [ ]:
feature.isna().sum()

In [ ]:
feature.info()

In [ ]:
feature.describe()

In [ ]:
train_data = feature.fillna({'Cross_Street':'Missing','Modus_Operandi':'Missing','Victim_Sex':'Missing','Victim_Descent':'Missing','Premise_Description':'Missing','Weapon_Description':'Missing'})
test = test_data.fillna({'Cross_Street':'Missing','Modus_Operandi':'Missing','Victim_Sex':'Missing','Victim_Descent':'Missing','Premise_Description':'Missing','Weapon_Description':'Missing'})

In [ ]:
train_data

In [ ]:
train_data.isna().sum()

In [ ]:
test.isna().sum()

In [ ]:
median_value = train_data['Weapon_Used_Code'].median()

# Fill missing values and assign it back to the DataFrame
train_data['Weapon_Used_Code'] = train_data['Weapon_Used_Code'].fillna(median_value)

In [ ]:
median_value = test['Weapon_Used_Code'].median()

# Fill missing values and assign it back to the DataFrame
test['Weapon_Used_Code'] = test['Weapon_Used_Code'].fillna(median_value)

In [ ]:
train_data.isna().sum()

In [ ]:
test.isna().sum()

In [ ]:
train_data.info()

In [ ]:
categorical = ['Location','Cross_Street','Date_Reported','Date_Occurred','Area_Name','Modus_Operandi','Victim_Sex','Victim_Descent','Premise_Description','Weapon_Description','Status','Status_Description']

In [ ]:
numerical = ['Latitude','Longitude','Time_Occurred','Reporting_District_no','Part 1-2','Victim_Age','Premise_Code','Weapon_Used_Code']

In [ ]:
correlation_matrix = data[numerical].corr()
plt.figure(figsize=(10, 6))
sns.heatmap(correlation_matrix, annot=True)
plt.title('Correlation Matrix')
plt.show()

In [ ]:
print(train_data.shape)

In [ ]:
# train_data = train_data.drop(columns=categorical)
# test_data = test_data.drop(columns=categorical)

In [ ]:
train_data.describe()

In [ ]:
train_data.info()

In [ ]:
train_data.isna().sum()

In [ ]:
numerical_features = train_data.select_dtypes(include=['int64', 'float64']).columns

print("Numerical Features:", numerical_features)

# Step 2: Apply Standard Scaling
scaler = StandardScaler()
train_data[numerical_features] = scaler.fit_transform(train_data[numerical_features])

In [ ]:
numerical_features = test.select_dtypes(include=['int64', 'float64']).columns

print("Numerical Features:", numerical_features)

# Step 2: Apply Standard Scaling
scaler = StandardScaler()
test[numerical_features] = scaler.fit_transform(test[numerical_features])

In [ ]:
def label_encode(df, columns):
    """
    Apply Label Encoding to specified columns in a DataFrame.
    
    Parameters:
    df (pd.DataFrame): The DataFrame containing the data.
    columns (list): A list of column names to apply Label Encoding to.
    
    Returns:
    pd.DataFrame: DataFrame with label encoded columns.
    dict: Dictionary of LabelEncoders used for each column.
    """
    label_encoders = {}
    
    for col in columns:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])
        label_encoders[col] = le  # Store the LabelEncoder for potential inverse transformation
        
    return df, label_encoders

In [ ]:
train_d , t_encoder = label_encode(train_data,categorical)
test_d , test_encoder = label_encode(test,categorical)

In [ ]:
test_d.isna().sum()

In [ ]:
train_d.isna().sum()

In [ ]:
test_d.info()

In [ ]:
train_d.info()

In [ ]:
train_d.describe()

In [ ]:
test_d.describe()

In [ ]:
# preprocessing = ColumnTransformer(
#     transformers=[
#         ('num', Pipeline(steps=[
#             ('imputer', SimpleImputer(strategy='median'))  # Fill NaNs with the median
#             ]),numerical) # Optionally, scale numerical data
#     ])

# # Create a Pipeline
# pipeline = Pipeline(steps=[
#     ('preprocessor', preprocessing)
    
# ])


# n_feature = pipeline.fit_transform(train_d)
# test = pipeline.fit_transform(test_d)

In [ ]:
# median_value = train_d['Weapon_Used_Code'].median()


# train_d['Weapon_Used_Code'] = train_d['Weapon_Used_Code'].fillna(median_value)

In [ ]:
# median_value = test_d['Weapon_Used_Code'].median()


# test_d['Weapon_Used_Code'] = test_d['Weapon_Used_Code'].fillna(median_value)

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(train_d,label,test_size=0.2,random_state=42)

In [ ]:
print(X_train.shape)
print(Y_test.shape)
print(Y_train.shape)
print(Y_test.shape)

**Decision Tree Model**

In [ ]:
dt_model = DecisionTreeClassifier(max_depth=6,min_samples_split=8)
dt_model.fit(X_train,Y_train)
y_pred = dt_model.predict(X_test)
accuracy = accuracy_score(Y_test,y_pred)
print(accuracy)
c_matrix = confusion_matrix(Y_test,y_pred)
print(c_matrix)

In [ ]:
lb = LabelEncoder()
# Fit on the training labels and transform
y_train = lb.fit_transform(Y_train)

# Transform the test labels using the fitted LabelBinarizer
y_test = lb.transform(Y_test)

In [ ]:
XGB_model = XGBClassifier(subsample = 0.8,reg_lambda = 0.001, n_estimators = 250,min_child_weight = 6,max_depth = 10,learning_rate = 0.1, gamma = 0.005,colsample_bytree = 0.5,)
XGB_model.fit(X_train,y_train)
y_pred = XGB_model.predict(X_test)
accuracy = accuracy_score(y_test,y_pred)
print(accuracy)



report = classification_report(y_test,y_pred)
print(report)
c_matrix = confusion_matrix(y_test,y_pred)
display = ConfusionMatrixDisplay(c_matrix, display_labels=np.unique(y_train))
display.plot(cmap='Blues')
plt.title('\nConfusion Matrix for ')
plt.show()

**Random Forest Classifier**

In [ ]:
rf_model = RandomForestClassifier(n_estimators=200,max_depth=6,min_samples_split=6)
rf_model.fit(X_train,Y_train)
y_pred = rf_model.predict(X_test)
accuracy = accuracy_score(Y_test,y_pred)
print(accuracy)
c_matrix = confusion_matrix(Y_test,y_pred)
display = ConfusionMatrixDisplay(c_matrix, display_labels=np.unique(Y_train))
display.plot(cmap='Blues')
plt.title('\nConfusion Matrix for ')
plt.show()

Logistic Regression Model

In [ ]:
lr_model = LogisticRegression(max_iter=500,)
lr_model.fit(X_train,Y_train)
y_pred = lr_model.predict(X_test)
accuracy = accuracy_score(Y_test,y_pred)
print(accuracy)
c_matrix = confusion_matrix(Y_test,y_pred)
display = ConfusionMatrixDisplay(c_matrix, display_labels=np.unique(Y_train))
display.plot(cmap='Blues')
plt.title('\nConfusion Matrix for ')
plt.show()

In [ ]:
# Plotting the learning curve
train_sizes, train_scores, test_scores = learning_curve(
    XGB_model, X_train, y_train, cv=5, n_jobs=-1, train_sizes=np.linspace(0.1, 1.0, 10)
)

# Calculate mean and standard deviation for train and test scores
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

plt.figure(figsize=(10, 6))
plt.title("Learning Curve for Random Forest Classifier")
plt.xlabel("Training examples")
plt.ylabel("Score")
plt.grid()

# Plotting the learning curve
plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, alpha=0.1,
                 color="r")
plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                 test_scores_mean + test_scores_std, alpha=0.1, color="g")
plt.plot(train_sizes, train_scores_mean, 'o-', color="r", label="Training score")
plt.plot(train_sizes, test_scores_mean, 'o-', color="g", label="Cross-validation score")

plt.legend(loc="best")
plt.show()


In [ ]:
# dummy_clf = DummyClassifier(strategy='most_frequent')
# dummy_clf.fit(feature,label)

In [ ]:
prediction = rf_model.predict(test_d)


In [ ]:
pred = lb.inverse_transform(prediction)

In [ ]:
pred

In [ ]:
# prediction

In [ ]:
submission=pd.DataFrame({'ID':range(1,5001),'total_amount':pred})
submission.to_csv('submission.csv',index=False)
print(submission)